# Practical 5: Putting it all together
<hr>
By now, you have seen several Python demos and progressed your coding abilities substantially. Therefore, it is safe and suitable for the fifth practical to focus more on exercises. This is what we will do. Past lecture sessions introduced relevant machine learning concepts, including tree-based models, neural networks, and the fundamental problem of overfitting. A lot of content to revisit. Let's go!

## Preliminaries
First of all, as usual, we import a couple of libraries. Also, we reproduce codes from previous sessions to prepare for subsequent tasks. In a nutshell, the following codes
- load the *California Housing Data Set*,
- partition the data set into a training (70%) and test set (30%)
- fit and assess a linear regression model on the training and test set, respectively

For the latter part, we introduce a custom function called `my_model_performance()` that takes care of the evaluation. Re-using this function will save you some time in Exercise 1. So take a little time to check out the function's code and how to use it. 

In [ ]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# SKlearn imports (all functions have been introduced in previous sessions)
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

#--------------------------------------------------------
# Loading the California Housing data set
#--------------------------------------------------------
calh = fetch_california_housing(as_frame=True)  # get the data as a Pandas dataframe
# separate the data into feature matrix X and target variable y
X = calh.data
y = calh.target
print('Loaded California Housing data set with dimension (rows x columns) {} x {}'.format(*X.shape))

#--------------------------------------------------------
# Data organization
#--------------------------------------------------------
test_frac = 0.3
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=test_frac)
print('{:.0f}% test set has dimension (rows x columns) {} x {}'.format(test_frac*100, *Xts.shape))

#--------------------------------------------------------
# Training (aka fitting) a linear regression model
#--------------------------------------------------------
lreg = LinearRegression().fit(Xtr, ytr)
print('Estimated coefficients of linear regression model')
df_print = pd.DataFrame(data=lreg.coef_, index=X.columns, columns=['Coefficient'])
print(df_print)

#--------------------------------------------------------
# Predict the test set and compute prediction performance
#--------------------------------------------------------
yhat_lreg = lreg.predict(Xts)

def my_model_performance(yhat, ytrue):
    ''' Helper function to assess the accuracy
        of forecasts in terms of MSE and MAE
        and format the results for printing.

        Inputs: yhat         (float) array with predictions
                ytrue        (float) array with observed outcomes

        Output: String; depicting model performance
    '''

    result = 'MSE={:.3f}, MAE={:.3f}'.format(
        mean_squared_error(y_true=yts, y_pred=yhat), 
        mean_absolute_error(y_true=yts, y_pred=yhat)
    )
    
    return result

print('Performance of linear regression model:\t', 
      my_model_performance(yhat_lreg, ytrue=yts)
     )


## Exercise 1: Warm-Up
Since the last coding sessions, we have heard about *Decision Trees* and *Neural Networks* but never really used them. Hence, this is what we do next. Here is your task: 

You can find the functionality to use the above learners in the modules `sklearn.neural_network` and  `sklearn.tree`. The corresponding classes are called, respectively, `MLPRegressor` and `DecisionTreeRegressor`. This information should suffice to train and assess your first neural network and tree model. More specifically, use the training data `Xtr` and `ytr`, which we created in the *Preliminaries* for training one network and one tree. Next, compute test set predictions and assess the models in the same way as illustrated above for linear regression using our custom function `my_model_performance()`.

In [ ]:
# Your solution to Exercise 1
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor

# Model training
dtree = DecisionTreeRegressor().fit(Xtr, ytr)
nnet = MLPRegressor().fit(Xtr, ytr)

# Model evaluation using our customer helper function
print('Performance of linear regression model:\t', 
      my_model_performance(lreg.predict(Xts), ytrue=yts)
     )
print('Performance of neural network model:\t', 
      my_model_performance(nnet.predict(Xts), ytrue=yts)
     )
print('Performance of decision tree model:\t', 
      my_model_performance(dtree.predict(Xts), ytrue=yts)
     )

Take a little time to examine model performance. Which model performs the best on this data set, the decision tree or the neural network? And do not forget to also compare to linear regression. There is no guarantee that more advanced learning algorithms give better results.

## Exercise 2: Overfitting
How did the lecture introduce overfitting? As the **fundamental problem** in machine learning. If that does not sound important, right? In theory, the decision tree and neural network model, which you just created should both be vulnerable to overfitting. However, we recommend you use the decision tree for Exercise #2 because it is faster to train. Here is your task:

Overfitting is a problem of complex machine learning models. The complexity of a decision tree depends, amongst others, on its depth. Explore the documentation of `DecisionTreeRegressor` and familiarize yourself with the argument `max_depth`. As the name suggests, this argument allows you to create decision tree models with different depths (i.e., number of splits). Train and assess multiple trees of increasing depth. Start with a depth of one and increase it incrementally. For each candidate depth, grow a tree of that depth and compute the corresponding test set error. Also compute the training set error and store these quantities. Then, create a plot that depicts how both errors, training and test error, develop when the depth of the tree increases. Based on the lecture, you should have a good idea of how the resulting plot should look like... 

<img src="https://raw.githubusercontent.com/stefanlessmann/ESMT_IML/main/resources/bias_var_overfitting.png" width="1280" height="720" alt="Bias, variance, and the fundamental problem of overfitting">

In [ ]:
# Your solution to Exercise 2
mdepths = range(1, 25)
mdepths[0]

mse_tr = []
mse_ts = []
for _, d in enumerate(mdepths):
    dtree = DecisionTreeRegressor(max_depth=d).fit(Xtr, ytr)
    mse_tr.append( mean_squared_error(ytr, dtree.predict(Xtr)) )
    mse_ts.append( mean_squared_error(yts, dtree.predict(Xts)) )

plt.plot(mdepths, mse_tr, label=['Training error'])
plt.plot(mdepths, mse_ts, label=['Test error'])
plt.xlabel('Maximum tree depth')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

At this point, you should look at an enticing visualization of the **overfitting problem**. The deeper the decision tree the more complex the model. More splits imply that the tree can model more complicated relationships between the features and the target. However, this ability can result in the tree fitting the *training* data too well. Once the tree starts to capture idiosyncratic noise during training and transfers this noise to its predictions, the error on the test set should increase. Therefore, your visualization highlights two important points. It provides you with empirical proof that overfitting is a real problem that you must be aware of. Further, and more practically, the plot can also suggest a suitable tree depth (i.e., meta-parameter setting) for this data set. The point where the test set error is the lowest marks the optimal depth for the tree for this data set. 

## Exercise 3: Benchmarking
Given that many different learning algorithms with different merits and demerits exist, and given that none of these should be considered generally better than others, a common scenario in machine learning is that one aims to find the most suitable approach for a given data set. Well, we are given a data set, right? So let's try to mimic this *benchmarking* scenario. To make it more interesting, we can further extend the set of learning algorithms we consider. The lecture has, although only briefly, mentioned two common tree-based ensemble algorithms, Random Forest and Gradient Boosting. You can easily access those via `sklearn`, which refers to them as `RandomForestRegressor` and `GradientBoostingRegressor`. These classes are available in the module `sklearn.ensemble`. 

For this exercise, you do not need to write much new code. Virtually all codes that you need have already appeared in one way or the other in previous parts of the notebook. Therefore, this exercise provides a great opportunity to further advance your coding skills. The idea of a benchmark is that you assess multiple learning algorithms. Each one undergoes a standardized treatment: train, compute predictions, compute performance, store results, etc. Surely there is a way to write the code for benchmarking several alternative learning algorithms in a suitable way without too much copy and pasting ;)

Here is your task:<be>
Implement a benchmarking experiment in which you assess the test set performance the following learning algorithms:
- Linear Regression
- Decision Tree
- Random Forest
- XGB
- Neural Network

Find out which one works best for the California Housing data set. To that end, plot the test set MAE of each model using a bar chart.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# Note that the configuration of some learning algorithms is chosen to ensure reasonably fast training
models = [LinearRegression(),
          MLPRegressor(hidden_layer_sizes=(5,)), 
          DecisionTreeRegressor(max_depth=8),  # based on Excercise 2, a max depth of 8 seems suitable
          RandomForestRegressor(n_estimators=5, max_depth=8), 
          GradientBoostingRegressor(n_estimators=10)]
model_keys = ['LR', 'MLP', 'DT', 'RF', 'GBM']  # these keys are mainly introduced for plotting and reporting results

dict_yhat = {}  # empty dictionary to store prediction
dict_mae = {}  # empty dictionary to store results in terms of MAE

# Loop over the learning algorithms
for i,m in enumerate(models):
    print('Fit model {}...'.format(m))  # letting the caller know where we are
    m.fit(Xtr, ytr)  # train model
    dict_yhat[model_keys[i]] = m.predict(Xts)  # predict test set and store the forecasts
    dict_mae[model_keys[i]] = mean_absolute_error(yts, dict_yhat[model_keys[i]])  # compute MAE and store the results

In [ ]:
# Create the bar plot
plt.barh(list(dict_mae.keys()), list(dict_mae.values()))
plt.xlabel('MAE')
plt.title('Benchmarking results')
plt.show()

## Exercise 4 (Extra): Hyperparameter tuning
Solving Exercise 3 is a major achievement. Several academic papers have been published on the basis of such benchmarks; so you may be proud to have made it up to this point!

In this additional exercise, we revisit some of the learning algorithms from the previous comparison and check whether we gave them fair treatment. More specifically, advanced learning algorithms like the neural network or gradient boosting exhibit several hyperparameters. For example, the number of boosting iterations is a hyperparameter in gradient boosting. Likewise, the architecture of the neural network needs to be determined by the data scientist. The fact that we did not specify any architecture above or did not specify the number of boosting iterations implies that all hyperparameters were set to default values. This approach is not appropriate. One should tune the hyperparameters of an algorithm to adjust it to a given data set. The `sklearn` library offers functionality for this task in the module `sklearn.model_selection`. This exercise is to familiarize yourself with some of the corresponding options for hyperparameter tuning.

Here is your task:
- Import the the function `GridSearchCV` from the module `sklearn.model_selection` and study its documentation.
- Consider the gradient boosting regressor.
    - Create a dictionary for tuning three of its hyperparameters, the number of base model trees (`n_estimators`), the maximum depth of individual decision trees (`max_depth`), and a third hyperparameter called the learning rate (`learning_rate`).
    - The following code snipped illustrates the structure of such dictionaries. We use the name of the hyperparameters (as shown above) as keys and set the corresponding values equal to lists of candidate settings for the hyperparameter. You can simply copy and paste the code into your solution.
- Call the `GridSearchCV` methods with the following specification:
    - Set the argument `param_grid` equal to your dictionary of candidate hyperparameter settings
    - Set the argument `estimator` equal to `GradientBoostingRegressor()`
    - Set the argument `cv` equal to 3
    - Set the argument `verbose` equal to 2
    - Make sure to store the result of the function by writing something like `searcher = GridSearchCV(....`
- Start the search for optimal hyperparameters by calling the well-known `fit()` function for the GridSearchCV object. For example, continuing the previous demo, your code could look like this: `searcher.fit()`.
- Note that running the `fit()` function may take a while. Once completed, inspect the result and determine which hyperparameters gave the best gradient boosting model, that is, delivered the lowest forecasting error.
- Compute test set predictions and their MAE using the best hyperparameter values. 
- Did hyperparameter tuning improve gradient boosting? Is the tuned model competitive to the learners of the Exercise 3 benchmark? 

**Demo of how to specify a search grid for hyperparameter tuning:** 
```
hyperparameter = {
    'n_estimators': [25, 50, 100],
    'max_depth': [2, 3, 5]
    'learning_rate' : [0.1, 0.01]
}
```

In [ ]:
from sklearn.model_selection import GridSearchCV
?GridSearchCV

In [ ]:
hyperparameter = {
    'n_estimators': [25, 25, 100],
    'max_depth': [2, 3, 5],
    'learning_rate' : [0.1, 0.01]
}

gbm = GridSearchCV(GradientBoostingRegressor(), 
                   param_grid=hyperparameter,
                   cv=3,
                   verbose=2
                  ).fit(Xtr, ytr)
gbm

In [ ]:
print('Performace of untuned GBM:\t', my_model_performance(ytrue=yts, yhat=dict_yhat['GBM']))
print('Performace of tuned GBM:\t', my_model_performance(ytrue=yts, yhat=gbm.predict(Xts)))
print('Performace of best benchmark:\t', my_model_performance(ytrue=yts, yhat=dict_yhat['RF'])) 

I assume your result shows that tuning has improved gradient boosting substantially and that the tuned boosting model outperforms all competitors with quite some margin. At least, this is the result I obtained when solving the tasks and running the notebook. After all, there is a reason for many data scientists calling gradient boosting *the best off the shelf learner*. Next time you seek a powerful learning algorithm for a regression or classification problem involving tabular data, pick gradient boosting, tune its hyperparameter to your data, this should give you a reasonably good idea of how well the target variable in your data can be predicted based on the available features. 



# Excellent Work! This was a challenging notebook, and you made it all through!